In [1]:
import numpy as np
import scipy as sp
import scipy.stats
from cvxopt import matrix, solvers

In [2]:
risk_free_rate = 0.02
time_to_maturity = 1
volatility = 0.3
strike = 100
stock_price = 100
n_trails = 1000
n_steps = 200
func_list = [lambda x: x**0, lambda x: x]

In [20]:
class MonteCarlo:
    def __init__(self,S0,K,T,r,sigma,underlying_process="geometric brownian motion"):
        self.underlying_process = underlying_process
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        
    def simulate(self, n_trails, n_steps, antitheticVariates=False):
        dt = self.T/n_steps
        self.n_trails = n_trails
        self.n_steps = n_steps
        if(self.underlying_process=="geometric brownian motion"):
#             first_step_prices = np.ones((n_trails,1))*np.log(self.S0)
            log_price_matrix = np.zeros((n_trails,n_steps))
            normal_matrix = np.random.normal(size=(n_trails,n_steps))
            if(antitheticVariates==True):
                n_trails *= 2
                self.n_trails = n_trails
                normal_matrix = np.concatenate((normal_matrix,-normal_matrix),axis=0)
            cumsum_normal_matrix = normal_matrix.cumsum(axis=1)
#             log_price_matrix = np.concatenate((first_step_prices,log_price_matrix),axis=1)
            deviation_matrix = cumsum_normal_matrix*self.sigma*np.sqrt(dt) + \
    (self.r-self.sigma**2/2)*dt*np.arange(1,n_steps+1)
            log_price_matrix = deviation_matrix+np.log(self.S0)
            price_matrix = np.exp(log_price_matrix)
            price_zero = (np.ones(n_trails)*self.S0)[:,np.newaxis]
            price_matrix = np.concatenate((price_zero,price_matrix),axis=1)
            self.price_matrix = price_matrix
        return price_matrix
    
    def BlackScholesPricer(self,option_type='c'):
        S = self.S0
        K = self.K
        T = self.T
        r = self.r
        sigma = self.sigma
        d1 = (np.log(S/K)+r*T +0.5*sigma**2*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        N = lambda x: sp.stats.norm.cdf(x)
        call = S * N(d1) - np.exp(-r*T) * K * N(d2)
        put = call - S + K*np.exp(-r*T)
        if(option_type=="c"):
            return call
        elif(option_type=="p"):
            return put
        else:
            print("please enter the option type: (c/p)")
        pass
    
    def MCPricer(self,option_type='c'):
        price_matrix = self.price_matrix
        # k = n_steps
        dt = self.T/self.n_steps
        df = np.exp(- self.r*dt)
        n_basis = len(func_list)
        n_trails = self.n_trails
        n_steps = self.n_steps
        
        if(option_type=="c"):
            payoff = (price_matrix[:,n_steps] - strike)
        elif(option_type=="p"):
            payoff = (strike - price_matrix[:,n_steps])
        else:
            print("please enter the option type: (c/p)")
            return
        
        payoff = matrix(np.where(payoff<0,0,payoff))
        vk = payoff*df
        regular_mc_price = np.average(payoff*np.exp(-risk_free_rate*time_to_maturity))
        self.mc_price = regular_mc_price
        return regular_mc_price
       
    def OHMCPricer(self,option_type='c', func_list=[lambda x: x**0, lambda x: x]):
        def _calculate_Q_matrix(S_k,S_kp1,df,func_list):
            dS = df*S_kp1 - S_k
            A = np.array([func(S_k) for func in func_list]).T
            B = (np.array([func(S_k) for func in func_list])*dS).T
            return np.concatenate((-A,B),axis=1)
        
        price_matrix = self.price_matrix
        # k = n_steps
        dt = self.T/self.n_steps
        df = np.exp(- self.r*dt)
        n_basis = len(func_list)
        n_trails = self.n_trails
        n_steps = self.n_steps
        
        if(option_type=="c"):
            payoff = (price_matrix[:,n_steps] - strike)
        elif(option_type=="p"):
            payoff = (strike - price_matrix[:,n_steps])
        else:
            print("please enter the option type: (c/p)")
            return
        
        payoff = matrix(np.where(payoff<0,0,payoff))
        vk = payoff*df
#         print("regular MC price",regular_mc_price)
    
        # k = 1,...,n_steps-1
        for k in range(n_steps-1,0,-1):
            Sk = price_matrix[:,k]
            Skp1 = price_matrix[:,k+1]
            Qk = matrix(_calculate_Q_matrix(Sk,Skp1,df,func_list))
            P = Qk.T * Qk
            q = Qk.T * vk
            A = matrix(np.ones(n_trails,dtype=np.float64)).T * Qk
            b = - matrix(np.ones(n_trails,dtype=np.float64)).T * vk
            sol = solvers.coneqp(P=P,q=q,A=A,b=b)
            ak = sol["x"][:n_basis]
            bk = sol["x"][n_basis:]
            vk = matrix(np.array([func(price_matrix[:,k]) for func in func_list])).T*ak*df
        
        # k = 0
        v0 = vk
        S0 = price_matrix[:,0]
        S1 = price_matrix[:,1]
        dS0 = df*S1 - S0
        Q0 = np.concatenate((-np.ones(n_trails)[:,np.newaxis],dS0[:,np.newaxis]),axis=1)
        Q0 = matrix(Q0)
        P = Q0.T*Q0
        q = Q0.T*v0
        A = matrix(np.ones(n_trails,dtype=np.float64)).T * Q0
        b = - matrix(np.ones(n_trails,dtype=np.float64)).T * v0
        C1 = matrix(ak).T * np.array([func(S1) for func in func_list]).T
        sol = solvers.coneqp(P=P,q=q,A=A,b=b)
        self.sol = sol
        residual_risk = (v0.T*v0 + 2*sol["primal objective"])/n_trails
        self.residual_risk = residual_risk[0]    # the value of unit matrix
        
        return sol["x"][0]
    
    
        
    def pricing(self, option_type='c', func_list=[lambda x: x**0, lambda x: x]):
        OHMC_price = self.OHMCPricer(option_type=option_type,func_list=func_list)
        regular_mc_price = self.MCPricer(option_type=option_type)
        black_sholes_price = self.BlackScholesPricer(option_type)
        return({"OHMC": OHMC_price,"regular MC": regular_mc_price,"Black-Scholes":black_sholes_price})
    
    def hedging(self):
        S = self.S0
        K = self.K
        T = self.T
        r = self.r
        sigma = self.sigma
        d1 = (np.log(S/K)+r*T +0.5*sigma**2*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        N = lambda x: sp.stats.norm.cdf(x)
        return({"OHMC optimal hedge": self.sol["x"][1],"Black-Scholes delta hedge":-N(d1),"OHMC residual risk":self.residual_risk})
        
    

In [21]:
mc = MonteCarlo(stock_price,strike,time_to_maturity,risk_free_rate,volatility)

In [22]:
price_matrix = mc.simulate(n_trails,n_steps,antitheticVariates=True)
mc.pricing()

{'Black-Scholes': 12.821581392691421,
 'OHMC': 13.345148910636944,
 'regular MC': 13.582320984915823}

In [23]:
price_matrix = mc.simulate(n_trails,n_steps,antitheticVariates=True)
mc.MCPricer('c')

12.156955274410789

In [5]:
price_matrix = mc.simulate(n_trails,n_steps)

In [6]:
prices = mc.pricing(func_list=func_list)
print(prices)

{'OHMC': 12.753994069215569, 'regular MC': 13.099527764158509, 'Black-Scholes': 12.821581392691421}
